In [30]:
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup as BS
from pprint import pprint
from lxml import html
from time import sleep

In [31]:
url = 'https://hh.ru/vacancies/data-analyst'

In [32]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',}

In [33]:
def razbor_str (stroka_in):
    
    if len(stroka_in) == 0:
        return [None,None,None]
    stroka_in = stroka_in.replace(' ', '')
    stroka_in = stroka_in.replace('\u202f', '')
    if 'руб.' in stroka_in:
        stroka_out = [None,None,'RU']
        stroka_in = stroka_in.replace('руб.', '')
    elif 'USD' in stroka_in:
        stroka_out = [None,None,'USD']
        stroka_in = stroka_in.replace('USD', '')
    elif 'EUR' in stroka_in:
        stroka_out = [None,None,'EUR']
        stroka_in = stroka_in.replace('EUR', '')    
    if 'от' in stroka_in:        
        stroka_out[0] = int(stroka_in[2:])
        stroka_out[1] = None
        return stroka_out
    elif 'до' in stroka_in:
        stroka_out[0] = None
        stroka_out[1] = int(stroka_in[2:])
        return stroka_out
        
    stroka_kort = stroka_in.split('–')      
    stroka_out[0] = int(stroka_kort[0])
    stroka_out[1] = int(stroka_kort[1])
    return stroka_out

In [35]:
params = {'page': 0}
pages = 3
n = 0
Total_list_Dict_vacancy =[]
Keys_Total_list_Dict_vacancy = ['_id','title','min','max','val','source']
while params['page'] <= pages:
    sleep(3)
    response = requests.get(url=url, headers=headers, params = params )
    dom = BS(response.text, 'html.parser')

    for link in dom.find_all('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}):
        templist = []
        templist.append(link['href'])
        templist.append(link.text)
        salary_str = razbor_str(link.parent.parent.parent.next_sibling.next_sibling.text)
        
        for i, v in enumerate(salary_str):
            templist.append(salary_str[i])
        templist.append('hh.ru')    
        Stroka_list_Dict_vacancy = dict(zip(Keys_Total_list_Dict_vacancy,templist))
        Total_list_Dict_vacancy.append(Stroka_list_Dict_vacancy)

    params['page'] += 1
    

In [38]:
#Добавление новых записей в базу Mango DB
client_ = MongoClient()
db = client.db_vacancy
for el in Total_list_Dict_vacancy:
    if db.collection_vacancy.find_one({'_id':el['_id']}) == None:
        try:
            db.collection_vacancy.insert_one(el)
        except DublicateKeyError:
            print(f'Дублирование id {el[_id]}')

In [39]:
print(len(list(db.collection_vacancy.find())))

194


In [42]:
curs = db.collection_vacancy.find()

In [43]:
doc = curs.next()
pprint(doc)

{'_id': 'https://spb.hh.ru/vacancy/69603794?query=data%20analyst',
 'max': None,
 'min': 150000,
 'source': 'hh.ru',
 'title': 'Product/Data Analyst',
 'val': 'RU'}


In [66]:
#Поиск зарплаты, удовлетворяющей условиям
salary = 1000
currency = 'USD'

for doc in db.collection_vacancy.find( { '$and': [ { 'min': { '$gte': salary } }, { 'val': currency } ] } ):
    pprint(doc)

{'_id': 'https://spb.hh.ru/vacancy/69635139?query=data%20analyst',
 'max': 4000,
 'min': 2000,
 'source': 'hh.ru',
 'title': 'Product Analyst (remote)',
 'val': 'USD'}
{'_id': 'https://spb.hh.ru/vacancy/69100086?query=data%20analyst',
 'max': 2000,
 'min': 1000,
 'source': 'hh.ru',
 'title': 'Junior Business Intelligence Analyst',
 'val': 'USD'}
{'_id': 'https://spb.hh.ru/vacancy/68620625?query=data%20analyst',
 'max': 4000,
 'min': 2000,
 'source': 'hh.ru',
 'title': 'Product Analyst (remote)',
 'val': 'USD'}
{'_id': 'https://spb.hh.ru/vacancy/69031296?query=data%20analyst',
 'max': 7000,
 'min': 3000,
 'source': 'hh.ru',
 'title': 'NLP Developer, Remote',
 'val': 'USD'}
{'_id': 'https://spb.hh.ru/vacancy/68736649?query=data%20analyst',
 'max': None,
 'min': 4000,
 'source': 'hh.ru',
 'title': 'Go Engineer (Data engineer)',
 'val': 'USD'}
{'_id': 'https://spb.hh.ru/vacancy/69450070?query=data%20analyst',
 'max': 4000,
 'min': 2500,
 'source': 'hh.ru',
 'title': 'Operations Manager Lis